### 按帧将视频切分成图片

In [43]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
# 定义保存图片函数
# image:要保存的图片
# pic_address：图片保存地址
# num: 图片后缀名，用于区分图片，int 类型

# 视频文件和图片保存地址
SAMPLE_VIDEO = './data/movie.mp4'
SAVE_PATH = './result/images/'

def normalization(data):
    _range = np.max(data) - np.min(data)
    return (data - np.min(data)) / _range
def normalize(arr):
    rng = arr.max()-arr.min()
    amin = arr.min()
    return (arr-amin)*255/rng
def save_image(image, address, num):
    pic_address = address + str(num) + '.jpg'
    imGray=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image=normalization(imGray)
    cv2.imwrite(pic_address, imGray)

def split_list_n_list(origin_list, n):
    if len(origin_list) % n == 0:
        cnt = len(origin_list) // n
    else:
        cnt = len(origin_list) // n + 1
 
    for i in range(0, n):
        yield origin_list[i*cnt:(i+1)*cnt][0]

def video_to_pic(video_path, save_path):
    # 读取视频文件
    frame_count = 0
    frame_record = []
    videoCapture = cv2.VideoCapture(video_path)
    # 读帧
    success, frame = videoCapture.read()
    frame_address=[]
    while success:
        frame_count = frame_count + 1
        # 每隔固定帧保存一张图片
        frame_record.append(frame)
        success, frame = videoCapture.read()
    frame_record=list(split_list_n_list(frame_record, 10))
    for i in range(len(frame_record)):
        save_image(frame_record[i], save_path, i)
        frame_address.append(f"{save_path}{i}.jpg")
    print('Save Success')
    return frame_address

if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)
frame_address=video_to_pic(SAMPLE_VIDEO, SAVE_PATH)

Save Success


### 计算Manhattan norm

In [44]:
from scipy.linalg import norm
def compare_images(img1, img2):
    # normalize to compensate for exposure difference, this may be unnecessary
    img1 = normalize(img1)
    img2 = normalize(img2)
    # calculate the difference and its norms
    diff = img1 - img2  # elementwise for scipy arrays
    m_norm = np.sum(abs(diff))  # Manhattan norm
    return m_norm
# read images as 2D arrays (convert to grayscale for simplicity)
img1 = cv2.imread('./result/images/1.jpg',0)
img2 = cv2.imread('./result/images/2.jpg',0)
# compare
n_m = compare_images(img1, img2)
print ("Manhattan norm:", n_m/img1.size)

Manhattan norm: 0.30087379418258714


### 计算Visual Variation

In [45]:
total_manhattan_norm=0
frame_nums=len(frame_address)
last_frame=[]
for i in range(frame_nums):
    # read images as 2D arrays (convert to grayscale for simplicity)
    if i==0:
        last_frame = cv2.imread(frame_address[0],0)
        continue
    img = cv2.imread(frame_address[i],0)
    # compare
    total_manhattan_norm += compare_images(last_frame, img)/img.size
    last_frame=img
print ("Visual Variation:", total_manhattan_norm/frame_nums)

Visual Variation: 0.2697731264068646


In [48]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
# 定义保存图片函数
# image:要保存的图片
# pic_address：图片保存地址
# num: 图片后缀名，用于区分图片，int 类型

# 视频文件和图片保存地址
SAMPLE_VIDEO = './data/movie.mp4'

def compare_images(img1, img2):
    # normalize to compensate for exposure difference, this may be unnecessary
    img1 = normalize(img1)
    img2 = normalize(img2)
    # calculate the difference and its norms
    diff = img1 - img2  # elementwise for scipy arrays
    m_norm = np.sum(abs(diff))  # Manhattan norm
    return m_norm
def normalization(data):
    _range = np.max(data) - np.min(data)
    return (data - np.min(data)) / _range
def normalize(arr):
    rng = arr.max()-arr.min()
    amin = arr.min()
    return (arr-amin)*255/rng
def split_list_n_list(origin_list, n):
    if len(origin_list) % n == 0:
        cnt = len(origin_list) // n
    else:
        cnt = len(origin_list) // n + 1
 
    for i in range(0, n):
        yield origin_list[i*cnt:(i+1)*cnt][0]

def video_to_pic(video_path):
    # 读取视频文件
    frame_count = 0
    frame_record = []
    videoCapture = cv2.VideoCapture(video_path)
    # 读帧
    success, frame = videoCapture.read()
    while success:
        frame_count = frame_count + 1
        # 每隔固定帧保存一张图片
        frame_record.append(frame)
        success, frame = videoCapture.read()
    frame_record=list(split_list_n_list(frame_record, 10))
    frame_record=[cv2.cvtColor(x, cv2.COLOR_BGR2GRAY) for x in frame_record]
    print('Save Success')
    return frame_record



frame_record=video_to_pic(SAMPLE_VIDEO)

total_manhattan_norm=0
frame_nums=len(frame_record)
last_frame=[]
for i in range(frame_nums):
    # read images as 2D arrays (convert to grayscale for simplicity)
    if i==0:
        last_frame = frame_record[0]
        continue
    img = frame_record[i]
    # compare
    total_manhattan_norm += compare_images(last_frame, img)/img.size
    last_frame=img
print ("Visual Variation:", total_manhattan_norm/frame_nums)

Save Success
Visual Variation: 0.26991686481961197
